In [1]:
import numpy as np
import os

In [11]:
test_words = ['one', 'two']
mfcc_path = 'mfccs_quantized'
train_mfcc_path = os.path.join(mfcc_path, 'training')
val_mfcc_path = os.path.join(mfcc_path, 'validation')
test_mfcc_path = os.path.join(mfcc_path, 'test')

n_mfcc = 20
ndim = 126

In [64]:
audio_path = os.path.join('..', 'speech_commands_quantized')
audio_length = 16000

In [10]:
train_count = 0
val_count = 0
test_count = 0
count = 0
for i in [train_mfcc_path, val_mfcc_path, test_mfcc_path]:
    for j in test_words:
        word_path = os.path.join(i, j)
        count += len(os.listdir(word_path))
    if (i == train_mfcc_path):
        train_count = count
    elif (i == val_mfcc_path):
        val_count = count
    else:
        test_count = count
    count = 0
print('Train count', train_count)
print('Validation count', val_count)
print('Test count', test_count)

Train count 6251
Validation count 696
Test count 823


In [13]:
train_mfcc = np.zeros(shape=(train_count, n_mfcc, ndim))
val_mfcc = np.zeros(shape=(val_count, n_mfcc, ndim))
test_mfcc = np.zeros(shape=(test_count, n_mfcc, ndim))

for i in [train_mfcc_path, val_mfcc_path, test_mfcc_path]:
    idx = 0
    for j in test_words:
        word_path = os.path.join(i, j)
        for k in os.listdir(word_path):
            file = os.path.join(word_path, k)
            mfcc = np.load(file)
            if (i == train_mfcc_path):
                train_mfcc[idx] = mfcc
            elif (i == val_mfcc_path):
                val_mfcc[idx] = mfcc
            elif (i == test_mfcc_path):
                test_mfcc[idx] = mfcc
            else:
                print('Something is wrong', i, j, k)
            idx += 1

In [28]:
cwd = os.getcwd()
np.save(os.path.join(cwd, 'train_mfcc'), train_mfcc)
np.save(os.path.join(cwd, 'val_mfcc'), val_mfcc)
np.save(os.path.join(cwd, 'test_mfcc'), test_mfcc)

In [36]:
#read in list of validation files
val_path = os.path.join(audio_path, 'validation_list.txt')
val_list = np.loadtxt(val_path, dtype = 'str')
val_list = [os.path.splitext(x)[0] for x in val_list]
display(val_list[:5])
print('Number of validation samples:', len(val_list))

['right/a69b9b3e_nohash_0',
 'right/439c84f4_nohash_1',
 'right/409c962a_nohash_1',
 'right/dbaf8fc6_nohash_2',
 'right/a6d586b7_nohash_1']

Number of validation samples: 9981


In [38]:
#read in list of test files
test_path = os.path.join(audio_path, 'testing_list.txt')
test_list = np.loadtxt(test_path, dtype = 'str')
test_list = [os.path.splitext(x)[0] for x in test_list]
display(test_list[:5])
print('Number of testing samples:', len(test_list))

['right/bb05582b_nohash_3',
 'right/97f4c236_nohash_2',
 'right/f2e59fea_nohash_3',
 'right/fdb5155e_nohash_2',
 'right/dc75148d_nohash_0']

Number of testing samples: 11005


In [63]:
train_audio_count = 0
val_audio_count = 0
test_audio_count = 0

for i in test_words:
    word_path = os.path.join(audio_path, i)
    for j in os.listdir(word_path):
        file = os.path.splitext(j)[0]
        file = i + '/' + file
        if file in test_list:
            test_audio_count += 1
        elif file in val_list:
            val_audio_count += 1
        else:
            train_audio_count += 1
print(train_audio_count == train_count)
print(val_audio_count == val_count)
print(test_audio_count == test_count)

True
True
True


In [96]:
train_audio = np.zeros(shape=(train_audio_count, audio_length))
val_audio = np.zeros(shape=(val_audio_count, audio_length))
test_audio = np.zeros(shape=(test_audio_count, audio_length))

val_label = np.empty(shape=(val_audio_count), dtype='<U10')
test_label = np.empty(shape=(test_audio_count), dtype='<U10')

In [97]:
train_idx = 0
val_idx = 0
test_idx = 0
for i in test_words:
    word_path = os.path.join(audio_path, i)
    for j in os.listdir(word_path):
        file_path = os.path.join(word_path,j)
        y = np.load(file_path)
        
        if (len(y) < audio_length):
            y = np.pad(y, (0, (audio_length - len(y))))
        
        file = os.path.splitext(j)[0]
        file = i + '/' + file
        if file in test_list:
            test_label[test_idx] = i
            test_audio[test_idx] = y
            test_idx += 1
        elif file in val_list:
            val_label[val_idx] = i
            val_audio[val_idx] = y
            val_idx += 1
        else:
            train_audio[train_idx] = y
            train_idx += 1

In [98]:
print(train_audio.shape)
print(val_audio.shape)
print(test_audio.shape)

print("\n", val_label.shape)
print(test_label.shape)

(6251, 16000)
(696, 16000)
(823, 16000)

 (696,)
(823,)


In [103]:
np.save(os.path.join(cwd, 'train_audio'), train_audio)
np.save(os.path.join(cwd, 'val_audio'), val_audio)
np.save(os.path.join(cwd, 'test_audio'), test_audio)

np.save(os.path.join(cwd, 'val_label'), val_label)
np.save(os.path.join(cwd, 'test_label'), test_label)